In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import os
import sys
from datetime import datetime, timedelta
from mysql.connector import connect, Error
from omegaconf import OmegaConf
import sqlalchemy
import pandas as pd
import boto3
from dotenv import load_dotenv

sys.path.append('../src')

In [33]:
current_date = (datetime.today() + timedelta(hours=3)).date()
date_start = current_date - timedelta(days=1)
date_start = datetime(
    date_start.year,
    date_start.month, 
    date_start.day
)

In [36]:
date_start < current_date

TypeError: can't compare datetime.datetime to datetime.date

In [35]:
date_start

datetime.datetime(2023, 1, 19, 0, 0)

In [20]:
load_dotenv()

True

In [21]:
conf = OmegaConf.load('../config.yaml')

In [22]:
mysql_host = os.getenv("MYSQL_HOST")
mysql_user = os.getenv("MYSQL_USER")
mysql_password = os.getenv("MYSQL_PASSWORD")
s3_access_key_id = os.getenv("S3_ACCESS_KEY_ID")
s3_secret_key = os.getenv("S3_SECRET_KEY")

id	channel_id	channel_name	date	text

In [10]:
# CONNECTION TO MYSQL SERVER
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password
    ) as connection:
        print(connection)
except Error as e:
    print(e)

ConfigAttributeError: Missing key host
    full_key: host
    object_type=dict

In [9]:
# CREATE DATABASE
db_query = "CREATE DATABASE gpb_news_external"
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(db_query)
except Error as e:
    print(e)

In [13]:
# SHOW DATABASES
query = "SHOW DATABASES"
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(query)
            for db in cursor:
                print(db)
except Error as e:
    print(e)

('MySQL-8300',)
('gpb_news_external',)
('information_schema',)


In [15]:
# CONNECTION TO EXISTING DATABASE
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
        database=conf.database
    ) as connection:
        print(connection)
except Error as e:
    print(e)

In [18]:
query = \
"""
CREATE TABLE telegram_news(
        id BIGINT,
        channel_id BIGINT,
        channel_name VARCHAR(100),
        date VARCHAR(100),
        text TEXT
)
"""
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
        database=conf.database
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(query)
            connection.commit()  # after any changes to table
except Error as e:
    print(e)

1050 (42S01): Table 'telegram_news' already exists


In [55]:
show_table_query = "DESCRIBE telegram_news"
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
        database=conf.database
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(show_table_query)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print(e)

('id', b'bigint', 'YES', '', None, '')
('channel_id', b'bigint', 'YES', '', None, '')
('channel_name', b'varchar(100)', 'YES', '', None, '')
('date', b'varchar(100)', 'YES', '', None, '')
('text', b'text', 'YES', '', None, '')


In [25]:
df = pd.read_json('../output/bbcrussian.json')
df.head()

,id,channel_id,channel_name,date,text
0,39132,1003921752,bbcrussian,2022-12-19 10:46:30+00:00,Военный корабль Королевских военно-морских сил...
1,39131,1003921752,bbcrussian,2022-12-19 10:25:37+00:00,
2,39130,1003921752,bbcrussian,2022-12-19 10:25:37+00:00,
3,39129,1003921752,bbcrussian,2022-12-19 10:25:36+00:00,
4,39128,1003921752,bbcrussian,2022-12-19 10:25:36+00:00,


In [60]:
try:
    connection = sqlalchemy.create_engine(f'mysql+mysqlconnector://{conf.user}:{conf.password}@{conf.host}/{conf.database}')
    df.to_sql(con=connection, name='telegram_news', if_exists='append', index=False)
except Error as e:
    print(e)

In [29]:
query = "SELECT * FROM telegram_news"
try:
    # with connect(
    #     host=conf.host,
    #     user=conf.user,
    #     password=conf.password,
    #     database=conf.database
    # ) as connection:
    connection = sqlalchemy.create_engine(f'mysql+mysqlconnector://{mysql_user}:{mysql_password}@{mysql_host}/{conf.db.database}')
    query_df = pd.read_sql(
        query, 
        connection,
        # chunksize=1000
    )
    # for chunk in query_df:
        # display(chunk)
except Error as e:
    print(e)

In [30]:
query_df.channel_name.value_counts()

meduzalive        103
bbcrussian         93
rian_ru            87
currenttime        62
radiosvoboda       50
varlamov_news      34
bazabazon          28
oldlentach         21
theinsider         21
novaya_pishet      13
nytimes             7
sharemed            7
washingtonpost      5
Name: channel_name, dtype: int64

In [31]:
query_df

,rownum,message_id,channel_id,channel_name,channel_url,date,text
0,1,40544,1003921752,bbcrussian,https://t.me/bbcrussian,2023-01-18 22:59:45,Что пишут западные СМИ о новом пакете помощи У...
1,2,40543,1003921752,bbcrussian,https://t.me/bbcrussian,2023-01-18 22:44:52,США выделяют 125 млн долларов на ремонт украин...
2,3,40542,1003921752,bbcrussian,https://t.me/bbcrussian,2023-01-18 22:22:01,Компания Microsoft уволит 10 тыс. своих сотруд...
3,4,40541,1003921752,bbcrussian,https://t.me/bbcrussian,2023-01-18 22:15:12,"Представитель Байдена: США не знают, что могло..."
4,5,40540,1003921752,bbcrussian,https://t.me/bbcrussian,2023-01-18 22:02:01,Суд на Филиппинах оправдал нобелевскую лауреат...
...,...,...,...,...,...,...,...
526,948,1956,1184664229,washingtonpost,https://t.me/washingtonpost,2023-01-19 23:23:45,Here is the latest from Ukraine:\n\n- U.S. Sec...
527,949,1955,1184664229,washingtonpost,https://t.me/washingtonpost,2023-01-19 21:35:03,Analysis: Putin says Russia’s economy beat exp...
528,950,1954,1184664229,washingtonpost,https://t.me/washingtonpost,2023-01-19 17:47:01,U.S. defense chief in Berlin for talks as Germ...
529,951,1953,1184664229,washingtonpost,https://t.me/washingtonpost,2023-01-19 14:51:01,U.S. readies another massive military package ...
